<a href="https://colab.research.google.com/github/dekhilons/Recommendation-engine-/blob/main/Demo_for_Startups_and_VCs_Recommendation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import ast
from langchain.schema import Document

# Load VC and Startup DataFrames
vc_df = pd.read_csv("/content/drive/MyDrive/Demo for Startups and VCs Recommendation Engine/vc_df.csv")
startup_df = pd.read_csv("/content/drive/MyDrive/Demo for Startups and VCs Recommendation Engine/startup_df.csv")

# Show a sample
print(vc_df.head())
print(startup_df.head())


            VC name  number of investment  \
0    StartUp Health                   390   
1           iAngels                   121   
2  Keiretsu Capital                   278   

                                         Description  \
0  StartUp Health is investing in a global army o...   
1  iAngels is a women-led venture capital firm an...   
2  Keiretsu Capital provides access to proprietar...   

                                             website last funding amount  \
0     https://www.linkedin.com/company/startuphealth         $19,300,000   
1           https://www.linkedin.com/company/iangels         $14,000,000   
2  https://www.linkedin.com/company/keiretsu-capital          $3,600,000   

  total funding amount  
0          $24,300,000  
1          $16,250,000  
2           $3,600,000  
  Company Name\t                      Industry   Headquarters  \
0     Perplexity  Artificial Intelligence (AI)  San Francisco   
1      Liquid AI  Artificial Intelligence (AI)      Cambrid

In [ ]:
def create_vc_text(row):
    """
    Combine the fields of a VC into a text block for embeddings.
    """
    description = row.get('Description', '')
    text_parts = [
        f"VC Name: {row.get('VC name', '')}",
        f"Website: {row.get('website', '')}",
        f"Number of Investments: {row.get('number of investment', '')}",
        f"Last Funding Amount: {row.get('last funding amount', '')}",
        f"Total Funding Amount: {row.get('total funding amount', '')}",
        f"Description: {description}"
    ]
    return "\n".join(text_parts)


def create_startup_text(row):
    """
    Combine the fields of a Startup into a text block for embeddings.
    """
    description = row.get('Description', '')
    full_description = row.get('Full Description', '')
    industry = row.get('Industry', '')
    foundation_year = row.get('Foundation year', '')

    text_parts = [
        f"Startup Name: {row.get('Company Name', '')}",
        f"Website: {row.get('Website', '')}",
        f"Industry: {industry}",
        f"Foundation Year: {foundation_year}",
        f"Description: {description}",
        f"Full Description: {full_description}"
    ]
    return "\n".join(text_parts)


In [ ]:
# Create VC Documents
vc_documents = []
for idx, row in vc_df.iterrows():
    content = create_vc_text(row)
    doc = Document(
        page_content=content,
        metadata={
            "vc_name": row.get('VC name', ''),
            "vc_website": row.get('website', '')
        }
    )
    vc_documents.append(doc)

# Create Startup Documents
startup_documents = []
for idx, row in startup_df.iterrows():
    content = create_startup_text(row)
    doc = Document(
        page_content=content,
        metadata={
            "startup_name": row.get('Company Name', ''),
            "startup_website": row.get('Website', '')
        }
    )
    startup_documents.append(doc)


In [ ]:
!pip install langchain chromadb sentence-transformers streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.6 MB/s eta 0:00:00

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Use the sentence-transformers model for embeddings
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
vc_embedder = HuggingFaceEmbeddings(model_name=MODEL_NAME)

# Create a Chroma vector store for VCs
vc_store = Chroma.from_documents(
    documents=vc_documents,
    embedding=vc_embedder,
    collection_name="vc_collection"
)

print("Chroma VC store created successfully!")


<ipython-input-6-d0fe09157f50>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vc_embedder = HuggingFaceEmbeddings(model_name=MODEL_NAME)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chroma VC store created successfully!


In [ ]:
def recommend_vcs_for_startup(startup_doc, vectorstore, top_k=5):
    """
    Given a startup document, return top-k most similar VCs from the vector store.
    """
    query_text = startup_doc.page_content
    results = vectorstore.similarity_search(query_text, k=top_k)
    return results


In [ ]:
# Test the recommendation engine with the first startup
sample_startup = startup_documents[0]  # Taking the first startup

results = recommend_vcs_for_startup(sample_startup, vc_store, top_k=3)

# Display the recommended VCs
print("Recommended VCs for Startup:")
for i, doc in enumerate(results):
    print(f"Rank #{i + 1}")
    print(f"VC Name: {doc.metadata['vc_name']}")
    print(f"VC Website: {doc.metadata['vc_website']}")
    print(f"Excerpt: {doc.page_content[:200]}...")
    print("-----")


Recommended VCs for Startup:
Rank #1
VC Name: iAngels
VC Website: https://www.linkedin.com/company/iangels
Excerpt: VC Name: iAngels
Website: https://www.linkedin.com/company/iangels
Number of Investments: 121
Last Funding Amount: $14,000,000
Total Funding Amount: $16,250,000
Description: iAngels is a women-led ven...
-----
Rank #2
VC Name: StartUp Health
VC Website: https://www.linkedin.com/company/startuphealth
Excerpt: VC Name: StartUp Health
Website: https://www.linkedin.com/company/startuphealth
Number of Investments: 390
Last Funding Amount: $19,300,000
Total Funding Amount: $24,300,000
Description: StartUp Healt...
-----
Rank #3
VC Name: Keiretsu Capital
VC Website: https://www.linkedin.com/company/keiretsu-capital
Excerpt: VC Name: Keiretsu Capital
Website: https://www.linkedin.com/company/keiretsu-capital
Number of Investments: 278
Last Funding Amount: $3,600,000
Total Funding Amount: $3,600,000
Description: Keiretsu C...
-----
